# Post-meet: Benter 1994 & NFL Data Project

## Data

**“game data.csv”**: Game level summary statistics from pro-football-reference.com. Data columns are:

- `link`: link to page data was scraped from, contains information on relevant team
- `week`: week of game
- `date`: date of game
- `ot`: indicator for if game went to overtime
- `rec`: record of team for that season after the game was completed
- `opp`: opposing team
- `team_score`: points scored
- `opp_score`: points allowed
- `o_fistdown`: first downs gained
- `o_yards`: total yards gained
- `o_passy`: pass yards gained
- `o_rushy`: rush yards gained
- `o_to`: offensive turnovers
- `d_fistdown`: first downs allowed
- `d_yards`: total yards allowed
- `d_passy`: pass yards allowed
- `d_rushy`: rush yards allowed
- `d_to`: defensive turnovers
- `ep_offense`: Offense’s “Expected Points”
- `ep_defense`: Defense’s “Expected Points”
- `ep_st`: Special Teams’ “Expected Points”
- `Team`: the home team (abbreviated)
- `Season`: season the game took place
- `eday`: date the game took place on
- `ha`: indicator if the “Team” is the home or away team
- `OddsOpen`: decimal odds of the “Team” winning at the start of betting (typically 6 days before the game is played)
- `OddsClose`: decimal odds of the “Team” winning right before kickoff
- `LineOpen`: Moneyline for the “Team” winning at the start of betting (typically 6 days before the game is played)
- `LineClose`: Moneyline for the “Team” winning right before kickoff
- `LineOddsOpen`: Payout for the moneyline bet (decimal odds) at start of betting for the “Team”
- `LineOddsClose`: Payout for the moneyline bet (decimal odds) right before kickoff
- `Opponent`: opposing team (abbreviated)
- `game_id`: identifier variable for the game. This is NOT a unique identifier as each game_id should be repeated twice. Observations should be unique by game_id and “Team”
- `take`: take or vigorish for the OddsClose bet
- `win`: 0/1 indicator for if the Team won or not

For all ELO measures see https://projects.fivethirtyeight.com/complete-history-of-the-nfl/

## Setup

In [67]:
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import os

# To avoid time conversion issues
os.environ['TZ'] = 'UTC'

In [68]:
class Bundle(dict):
    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(key)

    def __setattr__(self, key, value):
        self[key] = value

In [69]:
filename = "Data Project NFL/clean NFL.csv"
# Load the data from the CSV file
raw = pd.read_csv(filename)
print(raw.shape)
raw.head().T

(3584, 38)


,0,1,2,3,4
week,1,2,3,4,5
date,September 07,September 14,September 18,September 28,October 05
ot,OT,NaN,NaN,NaN,NaN
rec,1-0,1-1,2-1,2-2,2-3
opp,New Orleans Saints,Cincinnati Bengals,Tampa Bay Buccaneers,Minnesota Vikings,New York Giants
team_score,37.0,10.0,56.0,28.0,20.0
opp_score,34.0,24.0,14.0,41.0,30.0
o_fistdown,28.0,19.0,26.0,23.0,20.0
o_yards,568.0,309.0,488.0,411.0,397.0
o_passy,445.0,212.0,344.0,288.0,307.0


## EDA (Exploratory Data Analysis)

In [70]:
raw.dtypes

week               int64
date              object
ot                object
rec               object
opp               object
team_score       float64
opp_score        float64
o_fistdown       float64
o_yards          float64
o_passy          float64
o_rushy          float64
o_to               int64
d_fistdown       float64
d_yards          float64
d_passy          float64
d_rushy          float64
d_to               int64
ep_offense       float64
ep_defense       float64
ep_st            float64
Team              object
season             int64
eday              object
ha                object
OddsOpen         float64
OddsClose        float64
LineOpen         float64
LineClose        float64
LineOddsOpen     float64
LineOddsClose    float64
Opponent          object
game_id            int64
elo              float64
elo_prob         float64
qbelo            float64
qbelo_prob       float64
take             float64
win                int64
dtype: object

In [71]:
nulls = raw.isna().sum().sort_values(ascending=False)
pd.Series({n: ", ".join(nulls[nulls == n].index.tolist()) for n in nulls.unique()})
# print(nulls)

3414                                                   ot
386     ep_st, LineOddsClose, LineClose, LineOpen, Odd...
0       qbelo_prob, elo_prob, elo, game_id, take, week...
dtype: object

In [72]:
raw.ot.value_counts(dropna=False)

NaN    3414
OT      170
Name: ot, dtype: int64

We expect `OT` to be a boolean flag, instead it's "OT" or None. Must encode correctly. But first let look at the rest of NaNs.

Then there's apparently a set of 386 records with no data for a lot of fields. Grouping by season, we can see they're all from the 2020 season:

In [73]:
raw.season.value_counts()

2014    512
2015    512
2016    512
2017    512
2018    512
2019    512
2020    512
Name: season, dtype: int64

In [74]:
raw.groupby(["season"]).apply(lambda x: x.isna().sum()).round(2).T.head(10)

season,2014,2015,2016,2017,2018,2019,2020
week,0,0,0,0,0,0,0
date,0,0,0,0,0,0,0
ot,490,470,486,484,482,494,508
rec,0,0,0,0,0,0,386
opp,0,0,0,0,0,0,0
team_score,0,0,0,0,0,0,386
opp_score,0,0,0,0,0,0,386
o_fistdown,0,0,0,0,0,0,386
o_yards,0,0,0,0,0,0,386
o_passy,0,0,0,0,0,0,386


Not _every_ game from 2020 season has no data, let's look at per-week stats:

In [75]:
raw[raw.season == 2020].groupby(["week"]).apply(lambda x: x.isna().sum()).round(2).T.tail(10)

week,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
LineOddsOpen,0,0,0,0,30,28,28,28,28,28,26,32,30,32,32,32,32
LineOddsClose,0,0,0,0,30,28,28,28,28,28,26,32,30,32,32,32,32
Opponent,0,0,0,0,30,28,28,28,28,28,26,32,30,32,32,32,32
game_id,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
elo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
elo_prob,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
qbelo,0,0,0,0,30,28,28,28,28,28,26,32,30,32,32,32,32
qbelo_prob,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
take,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
win,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


From week 4 onwards, the data is missing, but the response variable `win` is never empty. Let's see its value distribution.

In [76]:
raw[(raw.season == 2020) & (raw.week >= 5)].win.value_counts(dropna=False)

0    386
Name: win, dtype: int64

No chance _both_ teams lost every match after week 5, this is a zero-value where a NaN value should have been. 

In [77]:
raw.groupby(raw.team_score.isna()).apply(lambda x: x.isna().mean()).round(2).T.sort_values(by=False, ascending=False).head(10)

team_score,False,True
ot,0.95,1.0
week,0.00,0.0
LineOddsClose,0.00,1.0
eday,0.00,0.0
ha,0.00,1.0
OddsOpen,0.00,1.0
OddsClose,0.00,1.0
LineOpen,0.00,1.0
LineClose,0.00,1.0
LineOddsOpen,0.00,1.0


Games with a value for `team_score`, only have `ot` missing (which we know already), and all other fields are present. _Every_ game without a `team_score` value, neither has any of the other missing fields, and although they have a `win` value, it's not significant.

**We'll drop every row without `team_score`, which should delete 386 rows:**


In [97]:
data = raw[raw.team_score.notna()].copy()
assert len(raw) - 386 == len(data)

In [98]:
data["ot"] = data["ot"].notna()

In [99]:
# Convert the 'eday' column to datetime dtype
data["raw_date"] = data.date
data['date'] = pd.to_datetime(data['eday'], format='%d%b%Y')
# Convert the datetime objects to Unix timestamps
data['ts'] = data.date.astype(int) // 10**9

data[["raw_date", "eday", "date", "ts"]].head()

,raw_date,eday,date,ts
0,September 07,07sep2014,2014-09-07,1410048000
1,September 14,14sep2014,2014-09-14,1410652800
2,September 18,18sep2014,2014-09-18,1410998400
3,September 28,28sep2014,2014-09-28,1411862400
4,October 05,05oct2014,2014-10-05,1412467200


In [100]:
from datetime import datetime

assert [
    datetime.fromtimestamp(timestamp) for timestamp in data.ts
] == data.date.tolist()

In [101]:
# Convert `rec` to three separate fields
# data[["win", "loss", "tie"]] = data['rec'].str.split("-",expand=True).fillna(0).astype(int)

In [102]:
assert sorted(data.ha.unique()) == ["A", "H"]  # Exclude any third options
data["H"] = data.ha == "H"

In [103]:
for x in ["o", "d"]:
    assert np.all(data[x + "_yards"] == data[x + "_passy"] + data[x + "_rushy"])

Even then, we'll keep the three and let the estimator decide on the better representation.

In [104]:
assert data[data.H].shape == data[~data.H].shape
data[data.H].shape, data[~data.H].shape

((1599, 41), (1599, 41))

In [105]:
data["diff_score"] = data.team_score - data.opp_score
data["signed_win"] = np.sign(data.diff_score)

In [106]:
data[["win", "signed_win"]].value_counts()

win  signed_win
0    -1.0          1592
1     1.0          1592
0     0.0            14
dtype: int64

## Feature Engineering

Every `week` of every `season`, several `games` between a `home_team` and an `away_team` happen. Some of the features available are particular to the match itself, including the betting odds and line odds:

In [107]:
stats_cols = [
    "diff_score",
    "team_score",
    "opp_score",
    "o_fistdown",
    "o_yards",
    "o_passy",
    "o_rushy",
    "o_to",
    "d_fistdown",
    "d_yards",
    "d_passy",
    "d_rushy",
    "d_to",
    "ep_offense",
    "ep_defense",
    "ep_st",
    "elo",
    "elo_prob",
    "qbelo",
    "qbelo_prob",
    "win",
]
betting_cols = [
    "OddsOpen",
    "OddsClose",
    "LineOpen",
    "LineClose",
    "LineOddsOpen",
    "LineOddsClose",
]

TODO: `stats_cols` include a lot of "post-game" metrics that you only know after the _end_ of the match, and cannot be used as features to predict the outcome (`[team|opp]_score, [o|d]_[yards|passy|rushy|fistdown|to]`), and some that look like "pre-game" stats about each team (`ep_[offense|defense|st], (qb)elo(_prob)`). We should clearly differentiate between the two, and keep the latter (instead of dropping them all) as prediction features.

For each match, we'll create two set of features, for the home (`home_`) and away (`away_`) teams. For each , match, "base feature" (i.e. each column in `stats_cols`) and home/away team, we'll calculate:
- the last value against any other team (`vsall_last_`),
- the historical average against any other team, up until last match included (`vsall_avg_`)
- the last value against the same opponent (`vsopp_last_`),
- the historical average against the same opponent, up until last match included (`vsopp_avg_`)
- the season average, up until last match included (`season_avg_`)

We do not include `season_last_` features, because except for the first match of the season it's identical to `vsall_last_`.


In [108]:
key = ["season", "week", "Opponent", "Team"]
data = data.sort_values(by=key)
vsall = data.groupby("Team", group_keys=False)[stats_cols]
vsopp = data.groupby(["Opponent", "Team"], group_keys=False)[stats_cols]
season = data.groupby(["season", "Team"], group_keys=False)[stats_cols]

In [125]:
feats = pd.concat(
    [
        data[key + betting_cols],
        # Last game stats
        vsall.shift().add_prefix("vsall_last_"),
        # Historical stats average (expanding.mean) up to last game (shift)
        vsall.apply(lambda x: x.shift().expanding().mean()).add_prefix("vsall_avg_"),
        # Sames as before, but exclusively against the current opponen
        vsopp.shift().add_prefix("vsopp_last_"),
        vsopp.apply(lambda x: x.shift().expanding().mean()).add_prefix("vsopp_avg_"),
        season.apply(lambda x: x.shift().expanding().mean()).add_prefix("season_avg_"),
    ],
    axis=1,
).set_index(key)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [ ]:
match_cols = key + ["date", "ts", "ot"]

In [ ]:
if "game_id" in data.columns:
    data = data.set_index("game_id").sort_index()
home, away = data[data.H], data[~data.H]
assert len(home) == len(away) == len(data) / 2
assert np.all(home.index == away.index)

In [133]:
home_feats = home[key].reset_index().set_index(key).join(feats).set_index("game_id").sort_index()
away_feats = away[key].reset_index().set_index(key).join(feats).set_index("game_id").sort_index()

In [134]:
X = home[match_cols].sort_index().join(
    home_feats.add_prefix("home_")
).join(away_feats.add_prefix("away_")).sort_index()

In [135]:
over_spread = np.array([
    (home.team_score + home.LineClose > home.opp_score).mean(),
    (away.team_score + away.LineClose > away.opp_score).mean(),
    (home.opp_score + away.LineClose > home.team_score).mean(),
]).round(3)
over_spread

array([0.467, 0.502, 0.502])

## Modelling

Benter:
> The type of model used by the author is the **multinomial logit** model proposed by Bolton and Chapman (1986). This model is well suited to horse racing and has the convenient property that its output is a **set of probability estimates which sum to 1 within each race**.

There are eight possible bets to make on every game:
$$ \text{Bet on the [money, spread] line for the [home, away] team @ book [open, close]}$$

To bet on any of those, we need an estimate of the probability of winning the event. There are several ways to do so. In no particular order, here are some possible models:

#### 1. "Trinomial" Signed Lines
For every "line" (money or spread), one of three things can happen: home wins, draw/push, away wins. One can then train such "trinomial" response (with a logit as Benter, or any other model that outputs probabilities for each class) for every line available (money, open spread, close spread), using the same set of observations, one for each match.
- Pros: very straightforward to implement
- Cons: loses information by not differentiating between a big (say, +20) and small (+2) win over the line.

#### 2. Parametric Point Scores
Instead of directly predicting bet's winning probabilities, we first explicitly model the point scoring distribution for both teams, and then derive a closed-form distribution for the point difference of every game. Maher (1982), Dixon & Coles (1997) do so for soccer games rather convincingly, representing the home and away scores essentially as Poisson RVs.
- Pros: allows for explicit mean/variance calculations about the game scores; provides ways of computing money, total and spread line odds.
- Cons: No obvious parametric distribution to fit even soccer scores (with +1 point increments), less so for american football.

#### 3. Score Difference PMF
Having no "total" line, the money and spread bets winning probabilities can be found knowing just the score difference distribution, which can be thought of as a discrete RV assuming integer values between `min_diff, max_diff`. Then, the probability of winning a bet with $L$ line, is
$$ P(diff > L) = \sum_{k>L} P(diff=k)$$
- Pros: Very few assumptions, any nonparametric classifier which outputs class probabilities would do.
- Cons: There are a _lot_ of possible lines. Even a [0.05, 0.95] quantile range is `[-22, 27]` in the dataset, for a total of 50 outcomes to train on just ~1.6K games

### 4. Score Difference CDF
Instead of finding the full PMF for the score difference $diff$ as in Model #3 ($P(diff=k)$ for every $k$), we could directly estimate the quantities of interest $P(diff > L) = 1 - F(L)$, where $F$ is the cumulative distribution function for $diff$. To that end, for every possible line $L$, we train a binary classifier, on the outcome $diff > L$, with every game available.
- Pros: Gives direct probabilities of winning any money/spread bet (money bets == spread bets with +-0 line).
- Cons: We need to train 50+ classifiers, one for each "step" on the CDF "ladder".

We'll go for Model #1, not because of any intrinsic merit, but due to a lack of time to explore other alternatives.

In [162]:
home.diff_score.describe(percentiles=[0.05, 0.95])

count    1599.000000
mean        1.821764
std        14.354579
min       -49.000000
5%        -22.000000
50%         3.000000
95%        27.000000
max        52.000000
Name: diff_score, dtype: float64

In [163]:
home.diff_score.value_counts().head(8)

 3.0     130
-3.0      94
 7.0      82
-7.0      67
 6.0      55
 10.0     51
-6.0      50
 14.0     48
Name: diff_score, dtype: int64

In [164]:
# "Signed" Responses
Y = pd.DataFrame(Bundle(
    win=np.sign(home.team_score - home.opp_score),
    open_spread=np.sign(home.team_score + home.LineOpen - home.opp_score),
    close_spread=np.sign(home.team_score + home.LineClose - home.opp_score),
))

In [169]:
Y.value_counts()

win   open_spread  close_spread
 1.0   1.0          1.0            662
-1.0  -1.0         -1.0            608
 1.0  -1.0         -1.0            159
-1.0   1.0          1.0             51
 1.0  -1.0          0.0             17
-1.0  -1.0          1.0             12
 1.0   0.0         -1.0             12
-1.0  -1.0          0.0             11
 1.0  -1.0          1.0             11
       1.0         -1.0             10
       0.0          0.0              9
       1.0          0.0              7
-1.0   1.0         -1.0              6
       0.0          0.0              4
                    1.0              4
 0.0  -1.0         -1.0              4
 1.0   0.0          1.0              4
-1.0   0.0         -1.0              3
 0.0   1.0          1.0              2
-1.0   1.0          0.0              2
 0.0  -1.0          1.0              1
dtype: int64

In [ ]:
Y = pd.DataFrame(
    {
        "sideome_win": home.team_score > home.opp_score,
        "away_win": away.team_score > away.opp_score,
        "draw": home.team_score == home.opp_score,
        "home_osp": home.team_score + home.LineOpen > home.opp_score, 
        "away_osp": away.team_score + away.LineOpen > away.opp_score, 
        "pusho": home.team_score + home.LineOpen == home.opp_score,
        "home_csp": home.team_score + home.LineClose > home.opp_score, 
        "away_csp": away.team_score + away.LineClose > away.opp_score,
        "pushc": home.team_score + home.LineClose == home.opp_score,
    }
    # "home_lo_win": home.team_score - home.opp_score > home.Line,
)

In [ ]:
X.shape, Y.shape

In [ ]:
first_season = data.season.min()
starting_elo = data[(data.season == first_season) & (data.week == 1)][["Team", "elo"]].set_index("Team").elo
team_rank = starting_elo.rank().astype(int)

In [ ]:
X["team_rank"] = team_rank[X.Team].values
X["opp_rank"] = team_rank[X.Opponent].values
# X["diff"] = home.team_score - home.opp_score

In [ ]:
X = X.drop(columns=["Team", "Opponent", "date"])


In [ ]:
assert (
    S.home_win + S.away_win + S.draw
    == S.home_osp + S.away_osp + S.pusho
    == S.home_csp + S.away_csp + S.pushc
    == len(data) / 2
)

### Model training

In [ ]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the historical match data for the team
# Split the data into training and testing sets
(
    X_train,
    X_test,
    win_train,
    win_test,
    open_spread_train,
    open_spread_test,
    close_spread_train,
    close_spread_test,
) = train_test_split(X, win, close_spread, close_spread, test_size=0.2, random_state=42)

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real, Integer

# Define the search space for the hyperparameters
search_space = {
    "num_leaves": Integer(2, 10),
    "learning_rate": Real(0.01, 0.25),
    "feature_fraction": Real(0.5, 1.0),
    "num_iterations": Integer(50, 500),
    "lambda_l1": Real(0, 10),
    "lambda_l2": Real(0, 10),
}

# Define the LightGBM parameters
params = {
    "objective": "multiclass",
    "num_class": 3,
    "metric": "multi_logloss",
    "boosting_type": "gbdt",
}

# Define the BayesSearchCV object for each model
search_params = Bundle(
    estimator=lgb.LGBMClassifier(**params),
    search_spaces=search_space,
    n_iter=10,
    cv=3,
    n_jobs=-1,
    scoring="neg_log_loss",  # "accuracy": pros and cons
    random_state=42,
)


In [ ]:
ys = ["win", "open_spread", "close_spread"]
models = Bundle({y: BayesSearchCV(**search_params) for y in ys})
models.win.fit(X_train, win_train)
models.open_spread.fit(X_train, open_spread_train)
models.close_spread.fit(X_train, close_spread_train)

In [ ]:
estimators = Bundle(
    win=models.win.best_estimator_.fit(X_train, win_train),
    open_spread=models.open_spread.best_estimator_.fit(X_train, open_spread_train),
    close_spread=models.close_spread.best_estimator_.fit(X_train, close_spread_train),
)

In [ ]:
preds = Bundle({res: est.predict(X_test) for res, est in estimators.items()})
probas = Bundle({res: est.predict_proba(X_test) for res, est in estimators.items()})

accuracies = Bundle(
    win=accuracy_score(win_test, win_pred),
    open_spread=accuracy_score(open_spread_test, open_spread_pred),
    close_spread=accuracy_score(close_spread_test, close_spread_pred),
)

In [ ]:
fis = Bundle(
    {
        name: pd.Series(est.feature_importances_, est.feature_name_).sort_values(
            ascending=False
        )
        for name, est in estimators.items()
    }
)

In [ ]:
plt.barh(fis.win[:20].index, fis.win[:20].values)

In [ ]:
plt.barh(fis.open_spread[:20].index, fis.open_spread[:20].values)

In [ ]:
dividends = X_test[[c for c in X.columns if "Odds" in c]]

In [ ]:
dividends.head()

In [ ]:
probs = pd.DataFrame().reindex_like(dividends)

In [ ]:
probs.home_OddsOpen = probs.home_OddsClose = probas.win[:, 0]
probs.away_OddsOpen = probs.away_OddsClose = probas.win[:, 2]  # col 1 is sign 0 or draw
probs.home_LineOddsOpen = probas.open_spread[:, 0]
probs.away_LineOddsOpen = probas.open_spread[:, 2]
probs.home_LineOddsClose = probas.close_spread[:, 0]
probs.away_LineOddsClose = probas.close_spread[:, 2]

probs.head().T.round(3)



In [ ]:
((probs * dividends) - 1).round(3).head(10).T

In [ ]:
scorecard = pd.DataFrame().reindex_like(dividends)


In [ ]:
scorecard.home_OddsOpen = scorecard.home_OddsClose = win[X_test.index] == 1
scorecard.away_OddsOpen = scorecard.away_OddsClose = win[X_test.index] == -1
scorecard.home_LineOddsOpen = open_spread[X_test.index] == 1
scorecard.away_LineOddsOpen = open_spread[X_test.index] == -1
scorecard.home_LineOddsClose = close_spread[X_test.index] == 1
scorecard.away_LineOddsClose = close_spread[X_test.index] == -1

scorecard.head(10).T

In [ ]:
eps = 0.01
bets = ((probs * dividends - 1) > eps).astype(int)
bets.head(10).T

In [ ]:
bets.values.sum()

In [ ]:
returns = (bets * dividends)[scorecard].fillna(0)

In [ ]:
rr = returns.T.reset_index(names="wager")
rr.groupby(rr.wager.str.contains("Open")).sum().sum(axis=1)

In [ ]:
pd.DataFrame(Bundle(bet_won=scorecard[bets == 1].mean(), pred_prob=probs[bets == 1].mean())).round(3)

In [ ]:
pd.DataFrame(Bundle(bet_won=scorecard[bets == 0].mean(), pred_prob=probs[bets == 0].mean())).round(3)

In [ ]:
import pandas as pd

identical_pairs = []
# Get every possible pair of column names from `data` using the `combinations` function
from itertools import combinations
pairs = combinations(X.columns, 2)

numeric_cols = X.select_dtypes(include=['number']).columns.tolist()
print(numeric_cols)
for col1, col2 in pairs:
    if col1 in numeric_cols and col2 in numeric_cols:
        if np.allclose(X[col1], X[col2]):
            identical_pairs.append((col1, col2, "num"))
        elif np.allclose(X[col1], -X[col2]):
            identical_pairs.append((col1, col2, "-num"))
    else:
        if X[col1].equals(X[col2]):
            identical_pairs.append((col1, col2, "obj"))

# Print the identical pairs of columns
print(*identical_pairs, sep="\n")

In [ ]:
X.filter(like="score").T.round(2)

In [ ]:
import seaborn as sns
sns.regplot(x=home.LineClose, y=-home.diff_score)

In [ ]:
(home.diff_score == -home.LineClose).mean()

In [ ]:
data[
    ["take"]
    + [(x + y) for x in ["Odds", "Line", "LineOdds"] for y in ["Open", "Close"]]
].describe(percentiles=[0.01, 0.05, 0.95, 0.99]).round(2)

## Bet sizing

In [ ]:
c = 0.06  # p hat
div = 20  # odds/dividend
er = c * div # expected return
ev = adv = er - 1  # advantage/expected value
pool = 100_000
vig = 0.0
wealth = pool / 2

In [ ]:
benter_bet = 416

In [ ]:
def breakeven_bet(pool, c, div, vig=0.0):
    return pool * (1 / (c * div) + vig - 1) / (1 - vig - 1/c)

In [ ]:
max_bet = breakeven_bet(pool, c, div, vig)
max_bet

In [ ]:
def expected_profit(bet, c, div, pool, vig):
    div_pre = div
    div_post = (pool + bet) * (1 - vig) / (pool / div_pre + bet)
    return (c * div_post - 1) * bet

In [ ]:
assert expected_profit(max_bet, c, div, pool, vig) == 0

In [ ]:
# Maximize expected profit with respect to `bet`, for c=0.06 and div0=20
from scipy.optimize import minimize_scalar
bounds = (0, max_bet)
res = minimize_scalar(lambda bet: -expected_profit(bet, c, div, pool, vig), bounds=bounds)
bet_opt, exp_profit = res.x, -res.fun
bounds, bet_opt, exp_profit, res

In [ ]:
bets = np.linspace(*bounds, 100)
ep = [expected_profit(b, c, div, pool, vig) for b in bets]

In [ ]:
expected_profit(bet_opt, c, div, pool, vig), exp_profit

In [ ]:
plt.plot(bets, ep)
# Draw a vertical line at `bet_opt`, with grey dashes
plt.axvline(bet_opt, color='grey', linestyle='--')
plt.axhline(exp_profit, color='grey', linestyle='--')
# Draw a vertical line at `benter_bet`, and a horizontal line at `expected_profit(benter_bet)`, both in orange and dashed
plt.axvline(benter_bet, color='orange', linestyle='--')
plt.axhline(expected_profit(benter_bet, c, div, pool, vig), color='orange', linestyle='--')

In [ ]:
# https://en.wikipedia.org/wiki/Kelly_criterion
kelly_bet = (c * div - 1) / (div - 1)  # adv / net_div
kelly_bet * wealth

In [ ]:
pd.Series(Bundle(
    benter=expected_profit(benter_bet, c, div, pool, vig),
    kelly=expected_profit(kelly_bet * wealth, c, div, pool, vig),
    kelly_third=expected_profit(2/3 * kelly_bet * wealth, c, div, pool, vig),
    ep_max=expected_profit(bet_opt, c, div, pool, vig),
)).round(2).sort_values(ascending=False)

In [ ]:
wiki_kelly = c - (1 - c) / (div -1)

In [ ]:
wiki_kelly, kelly_bet

In [ ]:
expected_profit(kelly_bet * wealth, c, div, pool, vig)

### Per-game betting

In [ ]:
home[["Team", "Opponent", "date", "ts"]].reset_index().plot("game_id", "date", kind="scatter")

In [ ]:
(home.date.sort_index().index.values, home.date.sort_values().index.values)

In [ ]:
# Add <1s to date timestamp in order to break ties
ts_ = home.ts + home.index.values / 1e6
# Make sure now the games sorted by ID match the games sorted by tie-broken timestamps
assert np.all(home.sort_index().index.values == ts_.sort_values().index.values)


In [ ]:
home.reset_index().groupby(["season"])[["date", "game_id"]].agg(["min", "max"])

#### Handicaps & Spreads

> You can bet on NFL games through three main bet types: the moneyline, total points, and handicap (spread) markets.
> The moneyline market is a straightforward bet on the winner of the game.
> The points total market is betting whether the game will go over or under a certain amount of points.
> The handicap market is where one side is given a point advantage so that both teams are close to even money, and this is where the big action on NFL betting is.
Source: https://insights.matchbook.com/betting-strategy/how-to-bet-on-the-nfl/

- `OddsOpen, OddsClose` have no other qualifier, so I'm guessing these are the "moneyline odds".
- `LineOpen, LineClose` have both positive and negative values, and the away's LineX is always exactly negative the home's LineX, so we'll assume this is the spread line and beta ccordingly. A Line bet, then, is won when `team_score + line >= opp_score`.


#### Betting sequence
- `game_ids` are monotonically increasing with the game date, so we'll assume we can bet sequentially on every game of the test set, ordered by game_id.


#### Bet sizing
- This is a terminating (finite amount of bets) game, so Kelly doesn't apply directly. To maximise EV(log(wealth)), in each bet we should bet our entire fortune on the highest +EV bet, if there's any. This is very silly, so we'll treat it as a nonterminating game and bet using Kelly.

- For every game, there are eight possible bets: [home, away] * [money, spread] * [open, close]. Since they need to be made simultaneously _and are correlated_, the "(fractional) Kelly" criterion is not immediately suitable. Given that Benter goes for "1/2 - 1/3" fractional Kelly, we'll go for an "sum-fractional Kelly", in which if there are `K` bets on a given game with +EV, we bet $1/(2K)\ to\ 1/(3K)$ on each one. Heuristic AF, but reasonable.
- To avoid (heuristically) 0-EV bets, we'll actually bet only on bets with $EV>\epsilon$, and set `eps=0.01`.
- Also, the form of Kelly betting which is usually known (eq. 5 in Benter) ignores any effect of bet on payoff odds. This might be true for NFL games with huge pots and individual bettors, but not for an organization in parimutuel betting markets, so we'll assume a pot size of `pool` dollars and calculate a profit-maximizing bet accordingly, as Benter. (NOTE: Why the diff wth benter's???)

So, given parameters:
- `wealth`
- `pool`
- `c`
- `div0`: the dividend (or `Odds`) _right before betting_
- `take`
- `kelly_frac`

Our bet will be
`min(ep_max_bet, kelly_bet)`
where
`kelly_bet := kelly_frac * adv / (div - 1) * wealth`
and 
`ep_max_bet := argmax{bet} expected_profit(bet, pool, c, div0, take)`
and the expected profit is `bet * (c * div1 - 1)`, where `div1` are the new odds, after accounting for the bet made.

https://www.forbes.com/betting/sports-betting/what-does-point-spread-mean/
What Does a Negative Spread Mean?

In the simplest terms, a negative spread indicates the favorite, which is the side that’s expected to win the matchup.


In [ ]:
def breakeven_bet(pool, c, div, vig=0.0):
    return pool * (1 / (c * div) + vig - 1) / (1 - vig - 1/c)


def expected_profit(bet, c, div, pool, vig):
    div_pre = div
    div_post = (pool + bet) * (1 - vig) / (pool / div_pre + bet)
    return (c * div_post - 1) * bet

def kelly_wealth_frac(c, div):
    # https://en.wikipedia.org/wiki/Kelly_criterion
    return (c * div - 1) / (div - 1)

In [ ]:
INITIAL_WEALTH = 1e6
EPS = 0.01  # epsilon
KELLY_RATIO = 1/2
POOL = 1e6
DEFAULT_TAKE = 0.05

In [ ]:
idx = 0
summary = pd.DataFrame(Bundle(
    divs=dividends.iloc[idx],
    probs=probs.iloc[idx],
))
summary["exp_ret"] = summary.divs * summary.probs
summary["place_bet"] = summary.exp_ret > (1 + eps)
placed_bets = summary.place_bet.sum()
placed_bets

In [ ]:
summary

In [ ]:
summary["kwf"] = np.vectorize(kelly_wealth_frac)(summary.probs, summary.divs)

In [ ]:
summary

In [ ]:
summary["kelly_bet"] = summary.kwf * wealth * kelly_ratio / placed_bets
summary["breakeven_bet"] = np.vectorize(breakeven_bet)(POOL, summary.probs, summary.divs, DEFAULT_TAKE)

In [ ]:
summary

In [ ]:
# Maximize expected profit with respect to `bet`, for c=0.06 and div0=20
from scipy.optimize import minimize_scalar
def epmax_bet(c, div, pool, vig, return_res=False):
    max_bet = breakeven_bet(pool, c, div, vig)
    if max_bet < 0:
        return None if return_res else np.nan, np.nan
    bounds = (0, max_bet)
    res = minimize_scalar(lambda bet: -expected_profit(bet, c, div, pool, vig), bounds=bounds)
    bet_opt, exp_profit = res.x, -res.fun
    return res if return_res else bet_opt, exp_profit

In [ ]:
np.vectorize(epmax_bet)(summary.probs, summary.divs, POOL, DEFAULT_TAKE)

In [ ]:
summary["epmax_bet"], summary["max_ep"] = np.where(
    summary.place_bet,
    np.vectorize(epmax_bet)(summary.probs, summary.divs, POOL, DEFAULT_TAKE),
    np.nan,
)

In [ ]:
summary["final_bet"] = np.minimum(summary.epmax_bet, summary.kelly_bet).fillna(0)

In [ ]:
summary["score"] = scorecard.iloc[idx]
summary["revenue"] = np.where(summary.score, summary.divs * summary.final_bet, -summary.final_bet)

wealth = INITIAL_WEALTH

In [ ]:
INITIAL_WEALTH

In [ ]:
INITIAL_WEALTH = 1e5
EPS = 0.01  # epsilon
KELLY_RATIO = 1/2
POOL = 1e6
DEFAULT_TAKE = 0.05
wealth = INITIAL_WEALTH
wealth_log = [wealth]
for game_id, divs in dividends.sort_index().head(200).iterrows():
    summary = pd.DataFrame(Bundle(
        divs=divs,
        probs=probs.loc[game_id],
        score=scorecard.loc[game_id],
    ))
    summary["exp_ret"] = summary.divs * summary.probs
    summary["place_bet"] = summary.exp_ret > (1 + eps)
    placed_bets = summary.place_bet.sum()
    summary["kwf"] = np.vectorize(kelly_wealth_frac)(summary.probs, summary.divs)
    summary["kelly_bet"] = summary.kwf * wealth * kelly_ratio / placed_bets
    summary["breakeven_bet"] = np.vectorize(breakeven_bet)(POOL, summary.probs, summary.divs, DEFAULT_TAKE)
    summary["epmax_bet"], summary["max_ep"] = np.where(
        summary.place_bet,
        np.vectorize(epmax_bet)(summary.probs, summary.divs, POOL, DEFAULT_TAKE),
        np.nan,
    )
    summary["final_bet"] = np.minimum(summary.epmax_bet, summary.kelly_bet).fillna(0)
    wealth -= summary.final_bet.sum()
    summary["revenue"] = np.where(summary.score, summary.divs * summary.final_bet, 0)
    # print(summary.round(2))
    wealth += summary.revenue.sum()
    wealth_log.append(wealth)
    print(f"{game_id}: {wealth:.2f}")


In [ ]:
plt.plot(wealth_log)
plt.yscale("log")
plt.show()

In [ ]:
INITIAL_WEALTH = 1e5
EPS = 0.01  # epsilon
KELLY_RATIO = 1/2
POOL = 1e6
DEFAULT_TAKE = 0.05
wealth = INITIAL_WEALTH
wealth_log = [wealth]
for game_id, divs in dividends.sort_index().head(200).iterrows():
    summary = pd.DataFrame(Bundle(
        divs=divs,
        probs=probs.loc[game_id],
        score=scorecard.loc[game_id],
    ))
    summary["exp_ret"] = summary.divs * summary.probs
    summary["place_bet"] = summary.score
    placed_bets = summary.place_bet.sum()
    summary["kwf"] = np.vectorize(kelly_wealth_frac)(summary.probs, summary.divs)
    summary["kelly_bet"] = summary.kwf * wealth * kelly_ratio / placed_bets
    summary["breakeven_bet"] = np.vectorize(breakeven_bet)(POOL, summary.probs, summary.divs, DEFAULT_TAKE)
    summary["epmax_bet"], summary["max_ep"] = np.where(
        summary.place_bet,
        np.vectorize(epmax_bet)(summary.probs, summary.divs, POOL, DEFAULT_TAKE),
        np.nan,
    )
    summary["final_bet"] = np.where(summary.score, 0.01 * wealth, 0)
    wealth -= summary.final_bet.sum()
    summary["revenue"] = np.where(summary.score, summary.divs * summary.final_bet, 0)
    # print(summary.round(2))
    wealth += summary.revenue.sum()
    wealth_log.append(wealth)
    print(f"{game_id}: {wealth:.2f}")


In [ ]:
plt.plot(wealth_log)
plt.yscale("log")
plt.show()

In [ ]:
INITIAL_WEALTH = 1e5
EPS = 0.01  # epsilon
KELLY_RATIO = 1/2
POOL = 1e6
DEFAULT_TAKE = 0.05
wealth = INITIAL_WEALTH
wealth_log = [wealth]
for game_id, divs in dividends.sort_index().head(200).iterrows():
    summary = pd.DataFrame(Bundle(
        divs=divs,
        probs=probs.loc[game_id],
        score=scorecard.loc[game_id],
    ))
    summary["exp_ret"] = summary.divs * summary.probs
    summary["place_bet"] = summary.score
    placed_bets = summary.place_bet.sum()
    summary["kwf"] = np.vectorize(kelly_wealth_frac)(summary.probs, summary.divs)
    summary["kelly_bet"] = summary.kwf * wealth * kelly_ratio / placed_bets
    summary["breakeven_bet"] = np.vectorize(breakeven_bet)(POOL, summary.probs, summary.divs, DEFAULT_TAKE)
    summary["epmax_bet"], summary["max_ep"] = np.where(
        summary.place_bet,
        np.vectorize(epmax_bet)(summary.probs, summary.divs, POOL, DEFAULT_TAKE),
        np.nan,
    )
    summary["final_bet"] = 0.01 * wealth
    wealth -= summary.final_bet.sum()
    summary["revenue"] = np.where(summary.score, summary.divs * summary.final_bet, 0)
    # print(summary.round(2))
    wealth += summary.revenue.sum()
    wealth_log.append(wealth)
    print(f"{game_id}: {wealth:.2f}")


In [ ]:
plt.plot(wealth_log)
plt.yscale("log")
plt.show()

In [ ]:
INITIAL_WEALTH = 1e5
EPS = 0.01  # epsilon
KELLY_RATIO = 1/2
POOL = 1e6
DEFAULT_TAKE = 0.05
wealth = INITIAL_WEALTH
wealth_log = [wealth]
for game_id, divs in dividends.sort_index().head(200).iterrows():
    summary = pd.DataFrame(Bundle(
        divs=divs,
        probs=probs.loc[game_id],
        score=scorecard.loc[game_id],
    ))
    summary["exp_ret"] = summary.divs * summary.probs
    summary["place_bet"] = summary.score
    placed_bets = summary.place_bet.sum()
    summary["kwf"] = np.vectorize(kelly_wealth_frac)(summary.probs, summary.divs)
    summary["kelly_bet"] = summary.kwf * wealth * kelly_ratio / placed_bets
    summary["breakeven_bet"] = np.vectorize(breakeven_bet)(POOL, summary.probs, summary.divs, DEFAULT_TAKE)
    summary["epmax_bet"], summary["max_ep"] = np.where(
        summary.place_bet,
        np.vectorize(epmax_bet)(summary.probs, summary.divs, POOL, DEFAULT_TAKE),
        np.nan,
    )
    summary["final_bet"] = np.where(summary.divs < 2, 0.01 * wealth, 0)
    wealth -= summary.final_bet.sum()
    summary["revenue"] = np.where(summary.score, summary.divs * summary.final_bet, 0)
    # print(summary.round(2))
    wealth += summary.revenue.sum()
    wealth_log.append(wealth)
    print(f"{game_id}: {wealth:.2f}")


In [ ]:
plt.plot(wealth_log)
plt.yscale("log")
plt.show()

In [ ]:
INITIAL_WEALTH = 1e5
EPS = 0.01  # epsilon
KELLY_RATIO = 1/2
POOL = 1e6
DEFAULT_TAKE = 0.05
wealth = INITIAL_WEALTH
wealth_log = [wealth]
for game_id, divs in dividends.sort_index().head(200).iterrows():
    summary = pd.DataFrame(Bundle(
        divs=divs,
        probs=probs.loc[game_id],
        score=scorecard.loc[game_id],
    ))
    summary["exp_ret"] = summary.divs * summary.probs
    summary["place_bet"] = summary.score
    placed_bets = summary.place_bet.sum()
    summary["kwf"] = np.vectorize(kelly_wealth_frac)(summary.probs, summary.divs)
    summary["kelly_bet"] = summary.kwf * wealth * kelly_ratio / placed_bets
    summary["breakeven_bet"] = np.vectorize(breakeven_bet)(POOL, summary.probs, summary.divs, DEFAULT_TAKE)
    summary["epmax_bet"], summary["max_ep"] = np.where(
        summary.place_bet,
        np.vectorize(epmax_bet)(summary.probs, summary.divs, POOL, DEFAULT_TAKE),
        np.nan,
    )
    summary["final_bet"] = np.where(np.random.uniform(size=len(summary)) > 0.5, 0.01 * wealth, 0)
    wealth -= summary.final_bet.sum()
    summary["revenue"] = np.where(summary.score, summary.divs * summary.final_bet, 0)
    # print(summary.round(2))
    wealth += summary.revenue.sum()
    wealth_log.append(wealth)
    print(f"{game_id}: {wealth:.2f}")


In [ ]:
plt.plot(wealth_log)
plt.yscale("log")
plt.show()

## Appendix: Some other stuff

### Better predicting

https://www.forbes.com/betting/sports-betting/what-does-point-spread-mean/
What Does a Negative Spread Mean?

In the simplest terms, a negative spread indicates the favorite, which is the side that’s expected to win the matchup.



In [ ]:
ys = ["win", "open_spread", "close_spread"]
models = Bundle({y: BayesSearchCV(**search_params) for y in ys})
models.win.fit(X_train, win_train)
models.open_spread.fit(X_train, open_spread_train)
models.close_spread.fit(X_train, close_spread_train)

### Distinguishing between "match metrics" (post-game) and "team stats" (pre-game)

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real, Integer

# Define the search space for the hyperparameters
search_space = {
    "num_leaves": Integer(2, 10),
    "learning_rate": Real(0.01, 0.25),
    "feature_fraction": Real(0.5, 1.0),
    "num_iterations": Integer(50, 500),
    "lambda_l1": Real(0, 10),
    "lambda_l2": Real(0, 10),
}

# Define the LightGBM parameters
# params = {
#     "objective": "binary",
#     "metric": "binary_logloss",
#     "boosting_type": "gbdt",
# }
params = {
    "objective": "multiclass",
    "num_class": 3,
    "metric": "multi_logloss",
    "boosting_type": "gbdt",
}
# Define the BayesSearchCV object for each model
search_params = Bundle(
    estimator=lgb.LGBMClassifier(**params),
    search_spaces=search_space,
    n_iter=27,
    cv=3,
    n_jobs=-1,
    scoring="neg_log_loss",  # "accuracy": pros and cons
    random_state=42,
)


In [ ]:
X = data[betting_cols]
# y = data.win
y = data.signed_win
model = BayesSearchCV(**search_params)
model.fit(X, y)
est = model.best_estimator_.fit(X, y)

In [ ]:
est

In [ ]:
truth = pd.get_dummies(data.signed_win)[[1.0, 0.0, -1.0]]
preds = est.predict_proba(X)

In [ ]:
np.log(preds[truth == 1]).sum()

In [ ]:
np.log(0.5) * len(truth)

In [ ]:
preds

In [ ]:
pd.DataFrame(preds)#.reindex_like(truth)

In [ ]:
(df.pred == df.true).mean(), df.pred.mean(), df.true.mean()

In [ ]:
pd.crosstab(est.predict(X), y)

In [ ]:
loglikelihood = 
def pseudoR2(preds, true)
    return 1 - (pred - true).var() / true.var()